In [ ]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install pymysql

In [10]:
# !pip install webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pymysql
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [23]:
def page_down():
    global error
    global end_crawling
    while True:
        print(100)
        driver.switch_to.window(driver.window_handles[0])  
        time.sleep(1)
        print(200)
        current_scroll = driver.execute_script('return window.pageYOffset')
        driver.execute_script(f'window.scrollTo(0, {current_scroll - 50});')
        time.sleep(1)
        print(300)
        driver.execute_script(f'window.scrollTo(0, {current_scroll + 150});')
        time.sleep(1)
        down_scroll = driver.execute_script('return window.pageYOffset')
        time.sleep(1)
        print(400)
        if current_scroll == down_scroll:
            print(500)
            driver.execute_script(f'window.scrollTo(0, {current_scroll - 50});')
            print(600)
            current_scroll = driver.execute_script('return window.pageYOffset')
            if current_scroll == down_scroll:
                print(600)
                error = True
                break
            else:
                print(700)
                end_crawling = True
                break
        try:
            print(800)
            driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{page_num}]/div/a/div[1]').click()          
            driver.switch_to.window(driver.window_handles[0])
            craw()
            print(900)
            driver.back()
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[0])  
            break
        except:
            print(1000)
            return page_down()
    return

In [24]:
def craw():
    for j in range(1,5):
        time.sleep(3)
        if j == 1:
            print(1)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobHeader_JobHeader__DBrFJ')
                
                name.append(element.text.split('\n')[0].split('∙')[0])
                loc.append(element.text.split('\n')[0].split('∙')[1])
                career.append(element.text.split('\n')[0].split('∙')[2])
                title.append(element.text.split('\n')[1])
            except:
                name.append('')
                loc.append('')
                career.append('')
                title.append('')
        elif j == 2:
            print(2)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'CompanyTags_CompanyTags__wpSCo')
                if len(element.text.split('\n')) >=1:
                    tag.append(element.text.split('\n')[1:])
                else:
                    tag.append('')    
            except:
                tag.append('')
        elif j == 3:
            print(3)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobDueTime_JobDueTime__iKbEO')
                if len(element.text.split('\n')) ==2:
                    timeout.append(element.text.split('\n')[1])
                else:
                    timeout.append('')
            except:
                timeout.append('')
        else:
            print(4)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobWorkPlace_JobWorkPlace__5fs5x')
                if len(element.text.split('\n')) == 2:
                    loc2.append(element.text.split('\n')[1])
                else:
                    loc2.append('')    
            except:
                loc2.append('')

In [25]:
def reset():
    global title
    global name
    global career
    global loc
    global tag
    global timeout
    global loc2
    global detail
    
    global done
    
    global current_scroll
    global down_scroll
    
    
    title = []           #채용공고
    name = []            #기업명
    career = []          #경력
    loc = []             #근무지
    tag = []             #태그
    timeout = []         #마감일
    loc2 = []            #근무지(상세)
    detail = []
    
    done = False

    # 스크롤 이동변수
    current_scroll = 0
    down_scroll = 0

In [26]:
def DF_make(file_name):
    df = pd.DataFrame()
    for i in tag:
         tag_str.append(','.join(i))

    df['기업명'] = name
    df['채용공고'] = title
    df['위치'] = loc
    df['위치상세'] = loc2
    df['태그'] = tag_str
    df['마감일'] = timeout
    df['경력'] = career

    df = df.drop_duplicates()
    df = df.fillna('')
    df.to_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_{file_name}.csv',encoding='utf-8')



In [27]:
def DB_insert(file_name):
    df = pd.read_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_{file_name}.csv',encoding='utf-8')
    conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
    cur = conn.cursor()
    cur.execute(f'''CREATE TABLE IF NOT EXISTS pro_{file_name} (
                   name VARCHAR(255),
                   title VARCHAR(255),
                   time VARCHAR(255),
                   loc VARCHAR(255),
                   career VARCHAR(255),
                   info VARCHAR(255),
                   loc2 VARCHAR(255)
                )''')
    
    for index, row in df.iterrows():
        cur.execute(f"""
            INSERT INTO pro_{file_name} (
               name,
               title,
               time,
               loc,
               career,
               info,
               loc2
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            row['기업명'],
            row['채용공고'],
            row['마감일'],
            row['위치'],
            row['경력'],
            row['태그'],
            row['위치상세']
        ))  

In [28]:
url_list = ['https://www.wanted.co.kr/wdlist/518/873?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/669?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/899?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/1634?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/10231?country=kr&job_sort=job.recommend_order&years=-1&locations=all',
            'https://www.wanted.co.kr/wdlist/518/1022?country=kr&job_sort=job.recommend_order&years=-1&locations=all']

url_name_list = ['wanted_web_developer',
                 'wanted_frontend_developer',
                 'wanted_python_developer',
                 'wanted_ML_engineer',
                 'wanted_data_engineer',
                 'wanted_data_scientist',
                 'wanted_DBA',
                 'wanted_BI_engineer']
file_url_name = dict(zip(url_list,url_name_list))

In [29]:
driver = webdriver.Chrome("")
time.sleep(10)
driver.get(page_url)
time.sleep(10)
error = False
page_num2 = 1
end_crawling = False
end_num = 100



for page_url,file_name in file_url_name.items():
    print('십육')
    reset()
    print('십칠')
    end_crawling = False
    error = False
    while True:
        print('십팔')
        try:
            for page_num in range(page_num2,end_num):
                if(page_num % 50) == 0:
                    print('십삼')
                    time.sleep(5)
                    driver.switch_to.window(driver.window_handles[0])
                    time.sleep(5)
                    print('십사')
                    driver.quit()
                    time.sleep(5)
                    driver = webdriver.Chrome("")
                    time.sleep(5)
                    print('십오')
                    driver.get(page_url)
                    time.sleep(5)
                try:
                    driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{page_num}]/div/a/div[1]').click()
                    driver.switch_to.window(driver.window_handles[0])
                    print('일')
                    craw()
                    print('이')
                    driver.back()
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    print('구')
                    page_down()
                if error == True:
                    print('십')
                    print(에러만들기)
                    print('십일')
                elif end_crawling == True:
                    print('십이')
                    break

        except:
            print('삼')
            time.sleep(5)
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(5)
            print('사')
            driver.quit()
            time.sleep(5)
            driver = webdriver.Chrome("")
            print('오')
            time.sleep(5)
            driver.get(page_url)
            time.sleep(5)
            print('육')
            page_num2 = page_num
            
        if (end_crawling ==True) or (page_num ==end_num):
            print('칠')
            DF_make(file_name)
#             DB_insert(file_name)
            page_num2 = 1
            print('팔')
            break

KeyboardInterrupt: 

In [ ]:
print(len(name))
print(len(title))
print(len(loc))
print(len(loc2))
print(len(tag))
print(len(timeout))
print(len(career))


# print(name)
print(title)
# print(loc)
# print(loc2)
# print(tag)
# print(timeout)
# print(career)